# Libraries

In [1]:
import os
import sys
import warnings
import pandas as pd
from lib.clipInterrogator import predict

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import KFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

IMAGES_DIR = "./images"
 
# Ignore warnings
warnings.filterwarnings("ignore")

models = {
  'Random Forest': RandomForestClassifier(n_estimators=100, random_state=0),
  'Extra Trees': ExtraTreesClassifier(n_estimators=100, random_state=0),
  'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=0),
  'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=0),
  'SVM': SVC(kernel='rbf', C=1, random_state=0),
}

C:\Users\Jorge Mora\AppData\Local\Temp\ipykernel_13388\55508.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Loading caption model blip-large...


c:\Users\Jorge Mora\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading CLIP model ViT-L-14/openai...
Loaded CLIP model and data in 4.66 seconds.


# Get Descriptors

In [2]:
import re
import unidecode
import nltk
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from nltk.stem.snowball import SnowballStemmer

# Download nltk data
nltk.download('punkt_tab')
nltk.download('stopwords')

# Get the stopwords in english
stop_words = set(stopwords.words('english'))

# Create a spell checker for English
spell = SpellChecker(language='en')

# Create a stemmer
stemmer = SnowballStemmer("spanish")

def removeSpecialChars(text):
  # Replace accented characters with their unaccented equivalents
  text = unidecode.unidecode(text)
  # Replace special characters with spaces
  text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
  # Delete multiple spaces
  text = re.sub(r'\s+', ' ', text).strip()

  return text

def removeStopWords(text):
  ## Remove stopwords
  words = text.split(" ")
  
  filtered_words = [word for word in words if word.lower() not in stop_words]

  return ' '.join(filtered_words)

def spellCheck(text):
  words = text.split(" ")

  for word in words:
    if word not in spell:
      candidates = spell.candidates(word)
      if candidates:
        word = list(candidates)[0]
  
  return " ".join(words)

def steamText(text):
  words = [stemmer.stem(word) for word in text.split(" ")]

  return " ".join(words)

def deleteDuplicates(text):
  words = text.split(" ")
  return " ".join(list(set(words)))

def processText(text):
  # Lower case
  text = text.lower()

  # Replace Special Charactors
  text = removeSpecialChars(text)

  # Remove StopWords
  text = removeStopWords(text)

  # SpellChecker
  text = spellCheck(text)

  # Delete again accented characters
  text = removeSpecialChars(text)

  # Stem text
  text = steamText(text)

  # Delete duplicates
  text = deleteDuplicates(text)

  return text

[nltk_data] Downloading package punkt_tab to C:\Users\Jorge
[nltk_data]     Mora\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Jorge
[nltk_data]     Mora\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.DataFrame(columns=['image', 'description', 'class'])

documents = []

for _class in os.listdir(IMAGES_DIR):
  print("Class:", _class)
  total = len(os.listdir(os.path.join(IMAGES_DIR, _class)))
  for i, img in enumerate(os.listdir(os.path.join(IMAGES_DIR, _class))):
    # Get prediction 
    descriptor = predict(os.path.join(IMAGES_DIR, _class, img), 'fast')

    descriptor = processText(descriptor)

    documents.append(descriptor)

    df.loc[len(df.index)] = [os.path.join(IMAGES_DIR, _class, img), descriptor, _class]
    sys.stdout.write(f"\rImage: {i + 1}/{total}")
    sys.stdout.flush()
  print()

Class: beach


100%|██████████| 55/55 [00:00<00:00, 211.58it/s]


Image: 1/538

100%|██████████| 55/55 [00:00<00:00, 192.67it/s]


Image: 2/538

...

Image: 536/538

100%|██████████| 55/55 [00:00<00:00, 139.33it/s]


Image: 537/538

100%|██████████| 55/55 [00:00<00:00, 156.60it/s]


Image: 538/538


# Create bag of words w/ TF-IDF

In [72]:
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=5000)

docs = vectorizer.fit_transform(documents)

columns = ['image', 'class'] + [f'feature_{i}' for i in range(docs.shape[1])]

data = []

for i, row in df.iterrows():
    vector = vectorizer.transform([row['description']]).toarray()[0]
    data.append([row['image'], row['class']] + list(vector))

_df = pd.DataFrame(data, columns=columns)

# Classify

In [73]:
_df = _df.drop('image', axis=1)
X = _df.drop("class", axis=1)
y = _df["class"]

In [76]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
  accuracies = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    accuracies.append(accuracy_score(y_test, y_pred))

  print(f"{name}:", np.mean(accuracies))

Random Forest: 0.9842032730404824
Extra Trees: 0.9869896640826873
Gradient Boosting: 0.9776959517657191
AdaBoost: 0.9823385012919896
SVM: 0.9842032730404824


# Mutual Info

In [77]:
mi = mutual_info_classif(docs, y)
feature_names = vectorizer.get_feature_names_out()

print("La mejor feature para clasificar es:", feature_names[np.argmax(mi)])

La mejor feature para clasificar es: beach
